In [1]:
# [+] PySpark 임포트 및 설정
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('key-value_rdd_operations')
sc = SparkContext(conf=conf)

### groupBy(): 입력 함수를 기준으로 그룹핑

In [2]:
# [+] 리스트로부터 rdd 생성 [1, 1, 2, 3, 5, 8]
rdd = sc.parallelize([1,1,2,3,5,8])

In [3]:
# [+] groupBy()를 이용한 홀수/짝수 그룹핑 -> groups
groups = rdd.groupBy(lambda x: x % 2)

In [4]:
groups

PythonRDD[5] at RDD at PythonRDD.scala:53

In [5]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()

### collect()
    rdd->list
    각각의 파티션(pc, node들)에 흩어져있는 data들을 collect()를 통해 master node에 모은다.
    원격으로 모으기에 구동 시간이 길다!

In [6]:
res

[(1, <pyspark.resultiterable.ResultIterable at 0x1c791fcc970>),
 (0, <pyspark.resultiterable.ResultIterable at 0x1c791f9bd90>)]

In [8]:
# [+] 그룹핑 결과(res) 출력
for k,v in res:
    print(k, list(v))

1 [1, 1, 3, 5]
0 [2, 8]


### groupByKey(): Key 값을 기준으로 그룹핑

In [9]:
# [+] 리스트로부터 key-value rdd 생성 [('a', 1), ('b', 1), ('a', 1)]
    # rdd.map(lambda 1, (x,1))
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)])

In [10]:
# [+] groupByKey()를 이용한 그룹핑 및 리스트 반환 -> groups
groups = rdd.groupByKey()

In [11]:
groups

PythonRDD[11] at RDD at PythonRDD.scala:53

In [12]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()

In [13]:
# [+]그룹핑 결과(res) 출력
for k,v in res:
    print(k, list(v))

a [1, 1]
b [1]


In [14]:
# len(): Python 객체의 길이를 리턴
#        Python 제공(내장 함수)
value = [1, 1]
len(value)

2

In [15]:
groups.count()
                # a와 b 하나씩 count

2

In [16]:
# mapValues(), len()을 이용한 값 빈도 계산
        # value에만 적용하는 함수
        # 각 항목에 존재하는 list의 길이만 
counts = groups.mapValues(len)

In [17]:
counts.collect()

[('a', 2), ('b', 1)]

### groupBy() 예제
1. 리스트로부터 RDD 객체 생성
    + 리스트: ['C', 'C++', 'Python', 'Java', 'C#']  
2. 원소의 첫 번째 문자 값을 기준으로 그룹핑
    + 기대 결과:  
        J ['Java']  
        C ['C', 'C++', 'C#']  
        P ['Python']  
3. 그룹핑 결과를 리스트 객체로 출력

In [18]:
# [+] 리스트로부터 RDD 객체 생성
rdd = sc.parallelize(['C', 'C++', 'Python', 'Java', 'C#'])

In [19]:
# [+] 원소별 첫 번째 값(x[0])을 기준으로 그룹핑 -> groups
groups = rdd.groupBy(lambda x : x[0])

In [20]:
# [+] 그룹핑 결과를 리스트로 반환 -> res
res = groups.collect()

In [21]:
# [+] 그룹핑 결과(res) 출력
for k,v in res:
    print(k, list(v))

C ['C', 'C++', 'C#']
P ['Python']
J ['Java']


### rdd에는 
    각 파티션에 흩어져 있다
    따라서 rdd 값 그대로 출력은 무용하다

### groupByKey() 예제
1. list 객체로부터 3개의 파티션을 갖는 Key-Value RDD 생성하기
    + getNumPatitions(): RDD 객체의 파티션 수 출력
2. Key 별로 그룹핑
3. 그룹핑 결과 리스트로 반환
4. 리스트 출력

In [22]:
# list 객체로부터 3개의 파티션을 갖는 Key-Value RDD 생성하기

rdd = sc.parallelize([  # 과목 별 점수 데이터
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3) # 파티션 수: 3
        # 12개 튜플을 파티션 3개로 쪼개 저장하겠다

In [23]:
# getNumPartitions(): RDD 객체의 파티션 수를 확인하는 액션 메서드
rdd.getNumPartitions()

3

In [24]:
# [+] Key 별로 그룹핑 -> groups
groups = rdd.groupByKey()

In [26]:
# [+] 그룹핑 결과 리스트로 반환 -> res
res = groups.collect()

In [27]:
# [+] 그룹핑 결과(res) 출력
for k,v in res:
    print(k, list(v))

Science [7, 6, 9, 5]
Math [7, 2, 8, 3]
English [7, 4, 9, 4]


In [28]:
# key별 그룹핑을 2개의 파티션으로 분리하여 수행 -> groups
groups = rdd.groupByKey(2)

In [29]:
# [+] 파티션 수 출력
groups.getNumPartitions()

2

In [30]:
# 파티션별 RDD 값들을 리스트로 반환
    # glom() = trans
    #     glom()이 있어야 파티션 별로 출력 가능
res = groups.glom().collect()

In [31]:
res

[[('English', <pyspark.resultiterable.ResultIterable at 0x1c7938034c0>),
  ('Science', <pyspark.resultiterable.ResultIterable at 0x1c793803dc0>)],
 [('Math', <pyspark.resultiterable.ResultIterable at 0x1c793803430>)]]

In [33]:
# res = groups.collect()
# res

[('English', <pyspark.resultiterable.ResultIterable at 0x1c793789160>),
 ('Science', <pyspark.resultiterable.ResultIterable at 0x1c793789c70>),
 ('Math', <pyspark.resultiterable.ResultIterable at 0x1c793803e20>)]

In [34]:
for k,v in res:
    print(k, list(v))

English [7, 4, 9, 4]
Science [7, 6, 9, 5]
Math [7, 2, 8, 3]


In [ ]:
for k,v in res:
    print(k, list(v))

In [112]:
# [+] 0번째 파티션에 대한 그룹핑 결과(res)를 출력


English [7, 4, 9, 4]
Science [7, 6, 9, 5]


In [113]:
# [+] 1번째 파티션에 대한 그룹핑 결과(res)를 출력


Math [7, 2, 8, 3]


### reduce() vs reduceByKey()
+ reduce(): 입력 함수를 기준으로 집계를 수행하는 액션 메서드
+ reduceByKey(): Key를 기준으로 그룹핑 및 집계를 수행하는 변환 메서드

In [37]:
# [+] reduce() 를 이용한 총합 구하기
sc.parallelize([1,2,3,4,5]).reduce(lambda x, y: x+y)

15

In [38]:
sc.parallelize([1,2,3,4,5]).reduce(lambda x, y: x*y)

120

In [41]:
# [+] 리스트로부터 K-V RDD 생성 -> rdd
rdd = sc.parallelize([('a',1), ('a',2), ('b',3), ('a',5),('b',7)])

In [42]:
# [+] reduceByKey()를 이용한 Key별 총합 계산 및 리스트로 반환
rdd.reduceByKey(lambda x,y : x+y).collect()

[('a', 8), ('b', 10)]

In [44]:
# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [45]:
# [+] reduceByKey() 를 이용한 과목 별 점수 총합 구하기
rdd.reduceByKey(lambda x,y: x+y).collect()

[('Science', 27), ('Math', 20), ('English', 24)]

In [46]:
# reduceByKey()는 transformation이므로 collect() 필수!!
rdd.reduceByKey(lambda x,y: x+y)

PythonRDD[63] at RDD at PythonRDD.scala:53

## 과목별 평균 구하기

In [48]:
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [50]:
scores_transformed = rdd.mapValues(lambda x: (x,1))
scores_transformed.collect() 

[('Math', (7, 1)),
 ('Math', (2, 1)),
 ('English', (7, 1)),
 ('Science', (7, 1)),
 ('English', (4, 1)),
 ('English', (9, 1)),
 ('Math', (8, 1)),
 ('Math', (3, 1)),
 ('English', (4, 1)),
 ('Science', (6, 1)),
 ('Science', (9, 1)),
 ('Science', (5, 1))]

In [47]:
# 과목별 평균 구하기
scores_reduced = scores_transformed.reduceByKey(lambda x,y: (x[0]+y[0]), (x[1]+y[1]))

PythonRDD[68] at RDD at PythonRDD.scala:53

### mapValues() vs map()
+ mapValues(): value에만 함수를 적용

In [51]:
# 리스트로부터 K-V RDD 객체 생성
rdd = sc.parallelize([
    ('a', ['apple', 'banana', 'lemon']),
    ('b', ['grapes'])
])

In [53]:
# mapValues(), len()를 이용하여 키 별 갯수 세기
rdd.mapValues(len).collect()

[('a', 3), ('b', 1)]

In [54]:
# map(): key, value 에 함수를 적용
    # (k-v) = x
rdd.map(len).collect()

[2, 2]

### countByKey()
Key별 값 개수를 계산 및 딕셔너리로 반환하는 액션 메서드

In [55]:
# countByKey()를 이용하여 Key별 값 배수 계산 및 반환 -> res
rdd = sc.parallelize([
    ('a', 1), ('b', 1), ('a', 1)
])

res = rdd.countByKey()
                        # action

In [56]:
# countByKey() 결과 출력
print(res)

defaultdict(<class 'int'>, {'a': 2, 'b': 1})


### keys()
key 목록을 RDD로 생성하는 변환 메서드

In [57]:
# 과목 별 점수 데이터
rdd = sc.parallelize([
    ('Math', 7), ('Math', 2), ('English', 7),
    ('Science', 7), ('English', 4), ('English', 9),
    ('Math', 8), ('Math', 3), ('English', 4),
    ('Science', 6), ('Science', 9), ('Science', 5)
], 3)

In [58]:
# RDD 키 목록 생성 및 출력

# rdd.keys() transformation
rdd.keys().collect()

['Math',
 'Math',
 'English',
 'Science',
 'English',
 'English',
 'Math',
 'Math',
 'English',
 'Science',
 'Science',
 'Science']

In [59]:
# [+] 유니크한 키 값 출력
                # distinct 사용
rdd.keys().distinct().collect()    

['Science', 'Math', 'English']

In [60]:
# [+] 유니크한 키 개수 출력
rdd.keys().distinct().count()

3

### Join 연산
+ join(): 내부조인
+ leftOuterJoin(): 외부조인(첫 번째 RDD 중심)
+ rightOuterJoin(): 외부조인(두 번째 RDD 중심)

In [61]:
# K-V RDD 객체 두 개 생성 
rdd1 = sc.parallelize([("a", 1), ("b", 4)])
rdd2 = sc.parallelize([("a", 2)])

In [62]:
# Join 연산 (key 기준)
rdd1.join(rdd2).collect()

[('a', (1, 2))]

In [64]:
# Left Outer Join 연산
rdd1.leftOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (1, 2))]

In [65]:
# Right Outer Join 연산
rdd1.rightOuterJoin(rdd2).collect()

[('a', (1, 2))]

In [69]:
# Full Outer Join
rdd1.fullOuterJoin(rdd2).collect()

[('b', (4, None)), ('a', (1, 2))]

In [67]:
rdd1.union(rdd2).collect()

[('a', 1), ('b', 4), ('a', 2)]